In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import tensorflow as tf
import numpy as np
import random
import json
import pickle
from keras import regularizers
from keras.models import load_model
from tkinter import *
import time
import tkinter.messagebox
import pyttsx3
import threading
import speech_recognition as sr
import os

nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEGION\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Load the intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
# Initialize lists to store words, classes, and documents
words = []
classes = []
documents = []

# List of characters to ignore in the patterns
ignore_letters = ['?', '!', '.', ',']

# Extract patterns and tags from intents data
for intent in intents['intents']:
    if 'patterns' in intent:  # Check if 'patterns' key exists
        for pattern in intent['patterns']:
            # Tokenize the pattern and store in words list
            word_list = word_tokenize(pattern)
            words.extend(word_list)

            # Append the tokenized pattern and its tag to the documents list
            documents.append((word_list, intent['tag']))

            # Append the tag to classes list, if it's not already present
            if intent['tag'] not in classes:
                classes.append(intent['tag'])
    else:
        print(f"Key 'patterns' not found in intent: {intent}")

In [4]:
# Lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Create training data
training = []
output = []
output_empty = [0] * len(classes)

# Create training set, bag of words for each sentence
for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# Shuffle features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype='object')

# Create training lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [5]:
# Define the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(20, input_shape=(len(train_x[0]),), activation='relu'))
model.add(tf.keras.layers.Dense(10, kernel_regularizer=regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=8, verbose=1)

# Save the model
model.save("chatbot_model.keras")

# Save the words and classes
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

Epoch 1/500


c:\Users\LEGION\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 843us/step - accuracy: 0.0470 - loss: 3.8470 
Epoch 2/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step - accuracy: 0.0718 - loss: 3.7814   
Epoch 3/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 0.0984 - loss: 3.7160
Epoch 4/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.1226 - loss: 3.6404
Epoch 5/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step - accuracy: 0.1188 - loss: 3.5458   
Epoch 6/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step - accuracy: 0.1228 - loss: 3.4117
Epoch 7/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step - accuracy: 0.1272 - loss: 3.2131
Epoch 8/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - accuracy: 0.2065 - loss: 2.9671
Epoch 9/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - accuracy: 0.2778 - loss: 2.7700
Epoch 10/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step - accuracy: 0.3459 - loss: 2.5756
Epoch 11/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 836us/step - accuracy: 0.3313 - loss: 2.5700
Epoch 12/500
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms

In [6]:
# Load model and other resources
model = load_model('chatbot_model.keras')
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Utility Methods
def clean_up_sentence(sentence):
    sentence_words = word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    for i in intents_json['intents']:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [8]:
class ChatInterface(Frame):

    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.master = master
        self.speech_mode = False  # Flag for speech mode

        # sets default bg for top level windows
        self.tl_bg = "#EEEEEE"
        self.tl_bg2 = "#EEEEEE"
        self.tl_fg = "#000000"
        self.font = "Verdana 10"

        menu = Menu(self.master)
        self.master.config(menu=menu, bd=5)
        
        # Menu bar
        # File
        menu.add_cascade(label="Clear Chat", command=self.clear_chat)

        # Option menu
        option_option = Menu(menu, tearoff=0)
        menu.add_cascade(label="Option", menu=option_option)
        option_option.add_command(label="Text Mode", command=self.set_text_mode)
        option_option.add_command(label="Speech Mode", command=self.set_speech_mode)

        self.text_frame = Frame(self.master, bd=6)
        self.text_frame.pack(expand=True, fill=BOTH)

        # scrollbar for text box
        self.text_box_scrollbar = Scrollbar(self.text_frame, bd=0)
        self.text_box_scrollbar.pack(fill=Y, side=RIGHT)

        # contains messages
        self.text_box = Text(self.text_frame, yscrollcommand=self.text_box_scrollbar.set, state=DISABLED,
                             bd=1, padx=6, pady=6, spacing3=8, wrap=WORD, bg=None, font="Verdana 10", relief=GROOVE,
                             width=10, height=1)
        self.text_box.pack(expand=True, fill=BOTH)
        self.text_box_scrollbar.config(command=self.text_box.yview)

        # frame containing user entry field
        self.entry_frame = Frame(self.master, bd=1)
        self.entry_frame.pack(side=LEFT, fill=BOTH, expand=True)

        # entry field
        self.entry_field = Entry(self.entry_frame, bd=1, justify=LEFT)
        self.entry_field.pack(fill=X, padx=6, pady=6, ipady=3)

        # frame containing send button
        self.send_button_frame = Frame(self.master, bd=0)
        self.send_button_frame.pack(fill=BOTH)

        # send button
        self.send_button = Button(self.send_button_frame, text="Send", width=5, relief=GROOVE, bg='white',
                                  bd=1, command=lambda: self.send_message_insert(None), activebackground="#FFFFFF",
                                  activeforeground="#000000")
        self.send_button.pack(side=LEFT, ipady=8)
        self.master.bind("<Return>", self.send_message_insert)

        # Add a button for speech input
        self.speech_button = Button(self.send_button_frame, text="Speak", width=5, relief=GROOVE, bg='white',
                                    bd=1, command=self.start_speech_thread, activebackground="#FFFFFF",
                                    activeforeground="#000000")
        self.speech_button.pack(side=LEFT, ipady=8)

    def play_response(self, response):
        engine = pyttsx3.init()
        engine.setProperty('rate', 120)
        engine.setProperty('volume', 1.0)
        engine.say(response)
        engine.runAndWait()

    def start_speech_thread(self):
        threading.Thread(target=self.capture_speech).start()

    def capture_speech(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            try:
                print("Listening...")
                audio = recognizer.listen(source)
                user_input = recognizer.recognize_google(audio)
                print(f"You said: {user_input}")
                self.entry_field.delete(0, END)
                self.entry_field.insert(0, user_input)
                self.send_message_insert(None)
            except sr.UnknownValueError:
                tkinter.messagebox.showerror("Speech Recognition", "Could not understand audio")
            except sr.RequestError as e:
                tkinter.messagebox.showerror("Speech Recognition", f"Could not request results; {e}")

    def last_sent_label(self, date):
        try:
            self.sent_label.destroy()
        except AttributeError:
            pass

        self.sent_label = Label(self.entry_frame, font="Verdana 7", text=date, bg=self.tl_bg2, fg=self.tl_fg)
        self.sent_label.pack(side=LEFT, fill=X, padx=3)

    def clear_chat(self):
        self.text_box.config(state=NORMAL)
        self.last_sent_label(date="No messages sent.")
        self.text_box.delete(1.0, END)
        self.text_box.config(state=DISABLED)

    def send_message_insert(self, event=None):
        user_input = self.entry_field.get()
        pr1 = "Human : " + user_input + "\n"
        self.text_box.configure(state=NORMAL)
        self.text_box.insert(END, pr1)
        self.text_box.configure(state=DISABLED)
        self.text_box.see(END)
        ob = chatbot_response(user_input)
        pr = "Friendlybot : " + ob + "\n"
        self.text_box.configure(state=NORMAL)
        self.text_box.insert(END, pr)
        self.text_box.configure(state=DISABLED)
        self.text_box.see(END)
        self.last_sent_label(str(time.strftime("Last message sent: " + '%B %d, %Y' + ' at ' + '%I:%M %p')))
        self.entry_field.delete(0, END)

        if self.speech_mode:
            t2 = threading.Thread(target=self.play_response, args=(ob,))
            t2.start()

    def set_text_mode(self):
        self.speech_mode = False
        tkinter.messagebox.showinfo("Mode Change", "Switched to Text Mode")

    def set_speech_mode(self):
        self.speech_mode = True
        tkinter.messagebox.showinfo("Mode Change", "Switched to Speech Mode")

window_size = "400x500"  # Define window size

root = Tk()
a = ChatInterface(root)
root.geometry(window_size)
root.title("Friendlybot")

# Set icon
icon_path = 'i.ico'  # Relative path to icon file
if not os.path.exists(icon_path):
    tkinter.messagebox.showwarning("Icon File Missing", f"The icon file {icon_path} was not found. Using default icon.")
else:
    root.iconbitmap(icon_path)

root.mainloop()

TclError: bitmap "i.ico" not defined